In [6]:

import spacy
import re
from spacy.language import Language

python -m venv .env
source .env/bin/activate
pip install -U spacy
python -m spacy download en_core_web_lg
The downloaded language model can be found at :

/usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_lg

In [ ]:
!python -m spacy download en_core_web_lg 

In [93]:
text='''From: M, Vikas <VikasM@tevera.com>\n
Sent: Friday, September 22, 2023 8:46 AM\n
To: M Mahendra <M.Mahendra1@kandi.com>; R Bellala <R.Bellala@kandi.com>; IT Claims Data Model Team <IBUClaimsModel@tevera.com>; Load Test Claims <LoadTest@tevera.com>\
Cc: DG-ADAI-Oracle-tevera-DBA<DG-ADAI-Oracle-tevera-DBA@kandi.com>; Vikram Mohit<Vikram.Mohit@kandi.com>\n
Subject: [EXTERNAL] RE: Old GRP's on Load Env\n
I told Suma to go ahead and drop the two GRPS (Green) as the LT database space has reached a critical level. Vikas Mehrunkar Manager, Engineering - Data Model |\
IBU Claims 5260 Western Avenue NW, Chevy Chase, MD 20815 Phone: (300)986-2427\n
ZjQcmQRYFpfptBannerStart\n
This Message Is From an External Sender 
This message came from outside your organization.\n
Vikas M Manager, Engineering - Data Model | IBU Claims 
5260 Western Avenue NW, Chevy Chase, MD 20815 \n
Phone: (002)986-247ext.24 | Mobile: (700)477-8427'''

In [67]:
nlp=spacy.load('en_core_web_sm')

ruler = nlp.add_pipe("entity_ruler",before='ner')
#List of Entities and Patterns (source: https://spacy.io/usage/rule-based-matching)
patterns = [
                #{"label": "email", "pattern": [{"TEXT": {"REGEX": "\S+@\S+"}}]},
                {"TEXT": {"REGEX": "(\d{3}[-.\s]?)?(\(\d{3}\)[-.\s]?)?\d{3}[-.\s]?\d{4}"}}
            ]
#ruler.add_patterns(patterns)

In [79]:
@Language.component('regex_matcher')
def regex_matcher(doc):
  expressions={
      "phone": re.compile(r"[\(]*(\d{3})\s*[\)-]\s*([\d-]+)(?:\s*ext[.\s]*(\d+))?\b"),
      "date": re.compile(r"(0[0-9]|1[0-9]|2[0-9]|3[0-1])[/](0[0-9]|1[0-2])[/]([1-2][0-9][0-9][0-9])"),
      "email": re.compile(r"\S+@\S+")                
  }
  spans=[]
  for label, expression in expressions.items():
      for match in re.finditer(expression, doc.text):
          start, end = match.span()
          entity = doc.char_span(start, end, label=label)
          if entity:
            spans.append(entity)

  doc.ents=list(doc.ents)+spacy.util.filter_spans(spans)
  return doc

 "phone": re.compile(r"(\d{3}[-.\s]?)?(\(\d{3}\)[-.\s]?)?\d{3}[-.\s]?\d{4}"),
      "date": re.compile(r"(0[0-9]|1[0-9]|2[0-9]|3[0-1])[/](0[0-9]|1[0-2])[/]([1-2][0-9][0-9][0-9])"),
      "email": re.compile(r"\S+@\S+")

In [81]:
#nlp = spacy.load('en_core_web_lg', disable = ['ner'])
nlp = spacy.load('en_core_web_lg')
#nlp = spacy.blank('en')
nlp.add_pipe('regex_matcher',before='ner')
#nlp.add_pipe('regex_matcher')
doc = nlp(text)

for ent in doc.ents:
    print(ent, ent.label_)
     
#20th last friday

Vikas ORG
<VikasM@tevera.com> email
Friday, September 22, 2023 8:46 AM DATE
<M.Mahendra1@kandi.com>; email
Bellala GPE
<R.Bellala@kandi.com>; email
<IBUClaimsModel@tevera.com>; email
<LoadTest@tevera.com>Cc: email
DG-ADAI-Oracle-tevera-DBA<DG-ADAI-Oracle-tevera-DBA@kandi.com>; email
Vikram PERSON
Mohit<Vikram.Mohit@kandi.com> email
Load Env ORG
Suma PERSON
two CARDINAL
LT ORG
Mehrunkar PERSON
Engineering - Data Model |IBU Claims 5260 Western Avenue NW ORG
Chevy Chase GPE
MD 20815 ORG
(300)986-2427 phone
Vikas M Manager ORG
Engineering - Data Model | IBU ORG
5260 CARDINAL
Chevy Chase GPE
MD 20815 ORG
(002)986-247ext.24 phone
(700)477-8427 phone


In [94]:
for ent in doc.ents:
    if (ent.label_=='phone') | (ent.label_=='email') | (ent.label_=='DATE'):
         text=text.replace(ent.text,'[External]')


In [95]:
print(text)

From: M, Vikas [External]

Sent: [External]

To: M Mahendra [External] R Bellala [External] IT Claims Data Model Team [External] Load Test Claims [External] [External] Vikram [External]

Subject: [EXTERNAL] RE: Old GRP's on Load Env

I told Suma to go ahead and drop the two GRPS (Green) as the LT database space has reached a critical level. Vikas Mehrunkar Manager, Engineering - Data Model |IBU Claims 5260 Western Avenue NW, Chevy Chase, MD 20815 Phone: [External]

ZjQcmQRYFpfptBannerStart

This Message Is From an External Sender 
This message came from outside your organization.

Vikas M Manager, Engineering - Data Model | IBU Claims 
5260 Western Avenue NW, Chevy Chase, MD 20815 

Phone: [External] | Mobile: [External]


In [74]:
namesSpcy=spacy.load(r'C:\Users\AdilFaizan\Desktop\Badges\model-best')

In [96]:
doc=namesSpcy(text)
for ent in doc.ents:
    text=text.replace(ent.text,'[External]')

In [97]:
print(text)

From: M, Vikas [External]

Sent: [External]

To: M [External] [External] R Bellala [External] IT Claims Data Model Team [External] Load Test Claims [External] [External] [External] [External]

Subject: [EXTERNAL] RE: Old GRP's on Load Env

I told [External] to go ahead and drop the two GRPS (Green) as the LT database space has reached a critical level. Vikas Mehrunkar Manager, Engineering - Data Model |IBU Claims 5260 Western Avenue NW, Chevy Chase, MD 20815 Phone: [External]

ZjQcmQRYFpfptBannerStart

This Message Is From an External Sender 
This message came from outside your organization.

Vikas M Manager, Engineering - Data Model | IBU Claims 
5260 Western Avenue NW, Chevy Chase, MD 20815 

Phone: [External] | Mobile: [External]
